In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import tensorflow_model_optimization as tfmot
import tensorflow_datasets as tfds

%load_ext tensorboard

from os import path
import pathlib
import tempfile

In [2]:
# normalizing the images to [0, 1]
def normalize(image, label):
    return tf.cast(image, tf.float32) / 255., label

def random_crop(image):
    cropped_image = tf.image.random_crop(
        image, size=[256, 256, 3])

    return cropped_image

def random_jitter(image):
    # resizing to 286 x 286 x 3
    image = tf.image.resize(image, [286, 286],
                          method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)

    # randomly cropping to 256 x 256 x 3
    image = random_crop(image)

    # random mirroring
    image = tf.image.random_flip_left_right(image)

    return image

def preprocess_flowers_train(image, label):
    image = random_jitter(image)
    return image, label

# -------------------------------

def preprocess_flowers(image, label):
    image = tf.image.resize(image, [256, 256],
                          method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
    return image, label

def load_flowers_dataset():  
    (ds_train, ds_validation, ds_test), ds_info = tfds.load(name="tf_flowers", 
                                             with_info=True,
                                             split=['train[:70%]', 'train[70%:85%]', 'train[85%:]'],  #70/15/15 split
                                             as_supervised=True)

    ds_train = ds_train.map(normalize)    
    ds_train = ds_train.map(preprocess_flowers)
    ds_train = ds_train.shuffle(ds_info.splits['train'].num_examples)
    
    ds_validation = ds_validation.map(normalize)
    ds_validation = ds_validation.map(preprocess_flowers)
    
    ds_test = ds_test.map(normalize)
    ds_test = ds_test.map(preprocess_flowers)
    
    return ds_train, ds_validation, ds_test

def load_beans_datasets():
    (ds_train, ds_validation, ds_test), ds_info = tfds.load(
        'beans',
        split=['train', 'validation', 'test'],
        shuffle_files=True,
        as_supervised=True,
        with_info=True,
    )
    
    ds_train = ds_train.map(normalize)
    ds_train = ds_train.shuffle(ds_info.splits['train'].num_examples)
    
    ds_validation = ds_validation.map(normalize)
    
    ds_test = ds_test.map(normalize)
    
    return ds_train, ds_validation, ds_test

# Optimization after training

## Quantization after training

1. Dynamic range quantization
2. Full integer quantization
3. Float16 quantization

### 1. Dynamic range quantization

Only the weights are converted from float to 8 bit int. 

### 2. Full integer quantization

Weights and activation outputs are quantizated. Good for microcontrolers and TPUs

In [6]:
def model_quantization(model_path, ds):
    # check if model was already in optimization folder
    if (len(model_path.split('/')[0].split('_')) == 3):
        optimized_dir = pathlib.Path(model_path.split('/')[0] + '/')
    else:
        optimized_dir = pathlib.Path(model_path.split('/')[0] + '_optimized/')
    model_name = model_path.split('/')[-1].split('.')[0]
    
    model = tf.keras.models.load_model(model_path)
    
    converter = tf.lite.TFLiteConverter.from_keras_model(model)
    tflite_model = converter.convert()
    
    # save converted tflite model
    tf_model_path = optimized_dir/(model_name + '.tflite')
    size = tf_model_path.write_bytes(tflite_model)
    print('Converted TFLite model ('+ str(size) +' Bytes) saved to: ' + str(tf_model_path))
    
    # 1. optimize model using dynamic range quantization
    converter.optimizations = [tf.lite.Optimize.DEFAULT]
    tflite_model_quant = converter.convert()
    
    tf_quant_model_path = optimized_dir/(model_name + '_dynamic_rage_quantization.tflite')
    size = tf_quant_model_path.write_bytes(tflite_model_quant)
    print('Dynamic range quantizatized TFLite model ('+ str(size) +' Bytes) saved to: ' + str(tf_quant_model_path))
    
    # 2. Full integer quantization
    def representative_data_gen():
        for input_value, _ in ds.batch(1).take(100):
            # Model has only one input so each data point has one element.
            yield [input_value]

    converter = tf.lite.TFLiteConverter.from_keras_model(model)
    converter.optimizations = [tf.lite.Optimize.DEFAULT]
    converter.representative_dataset = representative_data_gen

    tflite_model_quant = converter.convert()
    tf_quant_model_path = optimized_dir/(model_name + '_full_integer_quantization.tflite')
    size = tf_quant_model_path.write_bytes(tflite_model_quant)
    print('Full integer quantizatized TFLite model ('+ str(size) +' Bytes) saved to: ' + str(tf_quant_model_path))
    
    # 2.1 Full integer quantization with input and output in integer too
    try:
        converter = tf.lite.TFLiteConverter.from_keras_model(model)
        converter.optimizations = [tf.lite.Optimize.DEFAULT]
        converter.representative_dataset = representative_data_gen
        converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
        converter.inference_input_type = tf.uint8
        converter.inference_output_type = tf.uint8

        tflite_model_quant = converter.convert()
        tf_quant_model_path = optimized_dir/(model_name + '_full_integer_quantization_integer_io.tflite')
        size = tf_quant_model_path.write_bytes(tflite_model_quant)
        print('Full integer quantizatized with integer io TFLite model ('+ str(size) +' Bytes) saved to: ' + str(tf_quant_model_path))
    except:
        print('ERROR: Failed Full integer quantizatized with integer io TFLite model')
        
    # 3. float16 quantization
    converter = tf.lite.TFLiteConverter.from_keras_model(model)
    converter.optimizations = [tf.lite.Optimize.DEFAULT]
    converter.target_spec.supported_types = [tf.float16]
    
    tflite_model_quant = converter.convert()
    tf_quant_model_path = optimized_dir/(model_name + '_float16_quantization.tflite')
    size = tf_quant_model_path.write_bytes(tflite_model_quant)
    print('float16 quantizatized TFLite model ('+ str(size) +' Bytes) saved to: ' + str(tf_quant_model_path))


## Pruning and fine-tuning

1. Prune model to different sparsity  ( tf uses magnitude-based pruning )
    1. ConstantSparsity - sparsity is kept constant during training.
    2. PolynomialDecay - the degree of sparsity is changed during training.
2. Fine-tune model

https://www.machinecurve.com/index.php/2020/09/29/tensorflow-pruning-schedules-constantsparsity-and-polynomialdecay/

In [7]:
BATCH_SIZE = 64
PRUNING_EPOCHS = 10
prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude

In [8]:
# Some layers cannot be pruned

def prune_prunable_layers(model, pruning_params):
    """returns model for pruning with avoided non prunable layers"""
    
    # Rescaling layer cannot be pruned
    def apply_pruning_to_prunable(layer):
        if isinstance(layer, tf.keras.layers.experimental.preprocessing.Rescaling) or isinstance(layer, tf.keras.layers.experimental.preprocessing.Normalization):
            return layer
        return prune_low_magnitude(layer, **pruning_params)
    model_for_pruning = tf.keras.models.clone_model(
                                model,
                                clone_function=apply_pruning_to_prunable,
                            )
    return model_for_pruning

In [9]:
def prune_model(model_path, batch_size, pruning_epochs, ds_train, ds_validation, sparsity):
     # check if model was already in optimization folder
    if (len(model_path.split('/')[0].split('_')) == 3):
        optimized_dir = pathlib.Path(model_path.split('/')[0] + '/')
    else:
        optimized_dir = pathlib.Path(model_path.split('/')[0] + '_optimized/')
    model_name = model_path.split('/')[-1].split('.')[0]
    
    model = tf.keras.models.load_model(model_path)
    
    sparsities = [0.5, 0.75, 0.9]
    
    ds_train = ds_train.batch(batch_size)
    ds_validation = ds_validation.batch(batch_size)
    
    ds_train.cache()
    ds_validation.cache()
    
    # get number of images
    num_images = 0
    for i in ds_train.as_numpy_iterator():
        num_images+=1

    end_step = np.ceil(num_images / batch_size).astype(np.int32) * pruning_epochs

    # Define pruning configuration
    pruning_params_constant = {
        'pruning_schedule': tfmot.sparsity.keras.ConstantSparsity(target_sparsity=sparsity,
                                                                    begin_step=0,
                                                                    end_step=end_step)
    }

    pruning_params_polynomial = {
        'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(initial_sparsity=0,
                                                                final_sparsity=sparsity,
                                                                begin_step=0,
                                                                end_step=end_step)
    }

    # Rescaling layer cannot be wrapped in prune low magnitude
    try:
        model_for_pruning_constant = prune_low_magnitude(model, **pruning_params_constant)
        model_for_pruning_polynomial = prune_low_magnitude(model, **pruning_params_polynomial)
    except:
        model_for_pruning_constant = prune_prunable_layers(model, pruning_params_constant) 
        model_for_pruning_polynomial = prune_prunable_layers(model, pruning_params_polynomial)


    # Compile models for pruning
    model_for_pruning_constant.compile(optimizer='adam',
                      loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                      metrics=['accuracy'])
    model_for_pruning_polynomial.compile(optimizer='adam',
                      loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                      metrics=['accuracy'])


    constant_log_dir = pathlib.Path("./tmp/" + model_name + '_ConstantSparsity' + str(int(sparsity*100)))
    constant_log_dir.mkdir(parents=True, exist_ok=True)
    polynomial_log_dir = pathlib.Path("./tmp/" + model_name + '_PolynomialDecay' + str(int(sparsity*100)))
    polynomial_log_dir.mkdir(parents=True, exist_ok=True)

    # Model callbacks
    constant_callbacks = [
        tfmot.sparsity.keras.UpdatePruningStep(),
        tfmot.sparsity.keras.PruningSummaries(log_dir=str(constant_log_dir))
    ]
    polynomial_callbacks = [
        tfmot.sparsity.keras.UpdatePruningStep(),
        tfmot.sparsity.keras.PruningSummaries(log_dir=str(polynomial_log_dir))
    ]

    # Fitting data
    model_for_pruning_constant.fit(ds_train,
                              validation_data=ds_validation,
                              epochs=pruning_epochs,
                              callbacks=constant_callbacks)

    model_for_pruning_polynomial.fit(ds_train,
                              validation_data=ds_validation,
                              epochs=pruning_epochs,
                              callbacks=polynomial_callbacks)

    # Save pruned models
    model_constant_path = optimized_dir/(model_name + '_ConstantSparsity' + str(int(sparsity*100)) + '.h5')
    model_for_export = tfmot.sparsity.keras.strip_pruning(model_for_pruning_constant)
    model_for_export.save(str(model_constant_path))
    print('saved ' + str(model_constant_path))

    model_polynomial_path = optimized_dir/(model_name + '_PolynomialDecay' + str(int(sparsity*100)) + '.h5')
    model_for_export = tfmot.sparsity.keras.strip_pruning(model_for_pruning_polynomial)
    model_for_export.save(str(model_polynomial_path))
    print('saved ' + str(model_polynomial_path))

## Weight clustering

In [10]:
cluster_weights = tfmot.clustering.keras.cluster_weights
CentroidInitialization = tfmot.clustering.keras.CentroidInitialization

In [11]:
# Some layers cannot be weight clustered

def cluster_clustred_layers(model, cluster_params):    
    # Rescaling layer cannot be pruned
    def apply_clustering_to_clusterable(layer):
        if model.layers[0] == layer or  model.layers[-1] == layer:
            return layer
        try:
            x = cluster_weights(layer, **cluster_params)
            return x
        except:
            return layer

    model_for_clustering = tf.keras.models.clone_model(
                                model,
                                clone_function=apply_clustering_to_clusterable,
                            )
    return model_for_clustering

In [12]:
def weight_cluster_model(model_path, batch_size, epochs, ds_train, ds_validation, number_of_clusters):
    """ Weight clustering on given moodel 
    note: cannot use for cycle in this function to do different number of clusters because of compatibility issues"""
     # check if model was already in optimization folder
    if (len(model_path.split('/')[0].split('_')) == 3):
        optimized_dir = pathlib.Path(model_path.split('/')[0] + '/')
    else:
        optimized_dir = pathlib.Path(model_path.split('/')[0] + '_optimized/')
    model_name = model_path.split('/')[-1].split('.')[0]
    
    model = tf.keras.models.load_model(model_path)
    
    ds_train = ds_train.batch(batch_size)
    ds_validation = ds_validation.batch(batch_size)
    
    ds_train.cache()
    ds_validation.cache()
    
    # get number of images
    num_images = 0
    for i in ds_train.as_numpy_iterator():
        num_images+=1

    end_step = np.ceil(num_images / batch_size).astype(np.int32) * epochs

    # Define weight clustering configuration
    cluster_params_kmeans = {
                                  'number_of_clusters': number_of_clusters,
                                  'cluster_centroids_init': CentroidInitialization.KMEANS_PLUS_PLUS
                                }

    # Rescaling layer cannot be wrapped in prune low magnitude
    try:
        model_for_clustering_kmeans = cluster_weights(model, **cluster_params_kmeans)
    except:
        model_for_clustering_kmeans = cluster_clustred_layers(model, cluster_params_kmeans)

    # Compile models for clustering
    model_for_clustering_kmeans.compile(optimizer='adam',
                      loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                      metrics=['accuracy'])

    # Fitting data        
    model_for_clustering_kmeans.fit(ds_train,
                              validation_data=ds_validation,
                              epochs=epochs)

    # Save pruned models
    model_kmeans_path = optimized_dir/(model_name + '_KMeansPlusPlus' + str(number_of_clusters) + '.h5')
    model_for_export = tfmot.clustering.keras.strip_clustering(model_for_clustering_kmeans)
    model_for_export.save(str(model_kmeans_path))
    print('saved ' + str(model_kmeans_path))

# optimze models

In [13]:
# datasets
flowers_datasets = load_flowers_dataset()
beans_datasets = load_beans_datasets()

### Note

Due to bad tensorflow optimization of calling fit function in a loop, calling prune_model in a loop is unusable.
Solution: don't use for loops
Issue: https://github.com/tensorflow/tensorflow/issues/34025

## 1. Prune base models

In [11]:
prune_model(model_path='flowers_models/MobileNetV2_flowers_model.h5', 
            ds_train=flowers_datasets[0], 
            ds_validation=flowers_datasets[1],
            batch_size=BATCH_SIZE, 
            pruning_epochs=PRUNING_EPOCHS,
            sparsity=0.5)
prune_model(model_path='flowers_models/MobileNetV2_flowers_model.h5', 
            ds_train=flowers_datasets[0], 
            ds_validation=flowers_datasets[1],
            batch_size=BATCH_SIZE, 
            pruning_epochs=PRUNING_EPOCHS,
            sparsity=0.75)
prune_model(model_path='flowers_models/MobileNetV2_flowers_model.h5', 
            ds_train=flowers_datasets[0], 
            ds_validation=flowers_datasets[1],
            batch_size=BATCH_SIZE, 
            pruning_epochs=PRUNING_EPOCHS,
            sparsity=0.9)

prune_model(model_path='flowers_models/EfficentNetB0_flowers_model.h5', 
            ds_train=flowers_datasets[0], 
            ds_validation=flowers_datasets[1], 
            batch_size=BATCH_SIZE, 
            pruning_epochs=PRUNING_EPOCHS,
            sparsity=0.5)
prune_model(model_path='flowers_models/EfficentNetB0_flowers_model.h5', 
            ds_train=flowers_datasets[0], 
            ds_validation=flowers_datasets[1], 
            batch_size=BATCH_SIZE, 
            pruning_epochs=PRUNING_EPOCHS,
            sparsity=0.75)
prune_model(model_path='flowers_models/EfficentNetB0_flowers_model.h5', 
            ds_train=flowers_datasets[0], 
            ds_validation=flowers_datasets[1], 
            batch_size=BATCH_SIZE, 
            pruning_epochs=PRUNING_EPOCHS,
            sparsity=0.9)

/home/macenpav/.conda/envs/ml/lib/python3.9/site-packages/tensorflow/python/keras/engine/base_layer.py:2281: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  warnings.warn('`layer.add_variable` is deprecated and '


Epoch 1/10
 6/41 [===>..........................] - ETA: 20s - loss: 0.3910 - accuracy: 0.8870WARNING:tensorflow:Callback method `on_train_batch_end` is slow compared to the batch time (batch time: 0.1501s vs `on_train_batch_end` time: 0.3413s). Check your callbacks.


41/41 [==============================] - 53s 602ms/step - loss: 0.3764 - accuracy: 0.8853 - val_loss: 4.8429 - val_accuracy: 0.3648
Epoch 2/10
41/41 [==============================] - 38s 521ms/step - loss: 0.1317 - accuracy: 0.9549 - val_loss: 5.1772 - val_accuracy: 0.3339
Epoch 3/10
41/41 [==============================] - 36s 504ms/step - loss: 0.0294 - accuracy: 0.9913 - val_loss: 5.2955 - val_accuracy: 0.3285
Epoch 4/10
41/41 [==============================] - 38s 542ms/step - loss: 0.0145 - accuracy: 0.9966 - val_loss: 5.2196 - val_accuracy: 0.3249
Epoch 5/10
41/41 [==============================] - 35s 514ms/step - loss: 0.0052 - accuracy: 1.0000 - val_loss: 4.9734 - val_accuracy: 0.3466
Epoch 6/10
41/41 [==============================] - 32s 510ms/step - loss: 0.0156 - accuracy: 0.9983 - val_loss: 4.1645 - val_accuracy: 0.3811
Epoch 7/10
41/41 [==============================] - 35s 513ms/step - loss: 0.0663 - accuracy: 0.9779 - val_loss: 2.1195 - val_accuracy: 0.6189
Epoch 8/10

41/41 [==============================] - 55s 584ms/step - loss: 0.1297 - accuracy: 0.9544 - val_loss: 3.9216 - val_accuracy: 0.6134
Epoch 2/10
41/41 [==============================] - 33s 509ms/step - loss: 0.1730 - accuracy: 0.9408 - val_loss: 5.0273 - val_accuracy: 0.5318
Epoch 3/10
41/41 [==============================] - 37s 534ms/step - loss: 0.1029 - accuracy: 0.9661 - val_loss: 7.6865 - val_accuracy: 0.4900
Epoch 4/10
41/41 [==============================] - 36s 523ms/step - loss: 0.0559 - accuracy: 0.9817 - val_loss: 8.5656 - val_accuracy: 0.4628
Epoch 5/10
41/41 [==============================] - 35s 526ms/step - loss: 0.0334 - accuracy: 0.9883 - val_loss: 7.2760 - val_accuracy: 0.4701
Epoch 6/10
41/41 [==============================] - 35s 535ms/step - loss: 0.0483 - accuracy: 0.9872 - val_loss: 7.6444 - val_accuracy: 0.5390
Epoch 7/10
41/41 [==============================] - 34s 531ms/step - loss: 0.2076 - accuracy: 0.9316 - val_loss: 10.9768 - val_accuracy: 0.4374
Epoch 8/1

41/41 [==============================] - 55s 577ms/step - loss: 1.2790 - accuracy: 0.6239 - val_loss: 4.0899 - val_accuracy: 0.2668
Epoch 2/10
41/41 [==============================] - 35s 524ms/step - loss: 0.4252 - accuracy: 0.8521 - val_loss: 4.0015 - val_accuracy: 0.2668
Epoch 3/10
41/41 [==============================] - 37s 539ms/step - loss: 0.2112 - accuracy: 0.9382 - val_loss: 4.3952 - val_accuracy: 0.2668
Epoch 4/10
41/41 [==============================] - 35s 521ms/step - loss: 0.1263 - accuracy: 0.9717 - val_loss: 4.6068 - val_accuracy: 0.2668
Epoch 5/10
41/41 [==============================] - 31s 521ms/step - loss: 0.0764 - accuracy: 0.9828 - val_loss: 4.8142 - val_accuracy: 0.2668
Epoch 6/10
41/41 [==============================] - 37s 518ms/step - loss: 0.1036 - accuracy: 0.9668 - val_loss: 5.1166 - val_accuracy: 0.2668
Epoch 7/10
41/41 [==============================] - 35s 522ms/step - loss: 0.0861 - accuracy: 0.9786 - val_loss: 5.4248 - val_accuracy: 0.2668
Epoch 8/10

...

41/41 [==============================] - 80s 896ms/step - loss: 0.0971 - accuracy: 0.9636 - val_loss: 2.5507 - val_accuracy: 0.6207
Epoch 2/10
41/41 [==============================] - 43s 738ms/step - loss: 0.0985 - accuracy: 0.9630 - val_loss: 2.8862 - val_accuracy: 0.6116
Epoch 3/10
41/41 [==============================] - 48s 762ms/step - loss: 0.1622 - accuracy: 0.9485 - val_loss: 2.7197 - val_accuracy: 0.6062
Epoch 4/10
41/41 [==============================] - 46s 748ms/step - loss: 0.0768 - accuracy: 0.9758 - val_loss: 1.7427 - val_accuracy: 0.6770
Epoch 5/10
41/41 [==============================] - 47s 762ms/step - loss: 0.0416 - accuracy: 0.9840 - val_loss: 1.5539 - val_accuracy: 0.7042
Epoch 6/10
41/41 [==============================] - 46s 772ms/step - loss: 0.0255 - accuracy: 0.9910 - val_loss: 1.4259 - val_accuracy: 0.7205
Epoch 7/10
41/41 [==============================] - 44s 771ms/step - loss: 0.0270 - accuracy: 0.9907 - val_loss: 1.5866 - val_accuracy: 0.6933
Epoch 8/10

41/41 [==============================] - 75s 886ms/step - loss: 1.2346 - accuracy: 0.5257 - val_loss: 1.7721 - val_accuracy: 0.2668
Epoch 2/10
41/41 [==============================] - 46s 781ms/step - loss: 0.8428 - accuracy: 0.6856 - val_loss: 1.8009 - val_accuracy: 0.2668
Epoch 3/10
41/41 [==============================] - 44s 761ms/step - loss: 0.5879 - accuracy: 0.8060 - val_loss: 1.7687 - val_accuracy: 0.2777
Epoch 4/10
41/41 [==============================] - 45s 769ms/step - loss: 0.3840 - accuracy: 0.8961 - val_loss: 1.5808 - val_accuracy: 0.4537
Epoch 5/10
41/41 [==============================] - 46s 773ms/step - loss: 0.2851 - accuracy: 0.9189 - val_loss: 2.1422 - val_accuracy: 0.4283
Epoch 6/10
41/41 [==============================] - 45s 777ms/step - loss: 0.2061 - accuracy: 0.9514 - val_loss: 1.7510 - val_accuracy: 0.4864
Epoch 7/10
41/41 [==============================] - 47s 768ms/step - loss: 0.1700 - accuracy: 0.9548 - val_loss: 1.1124 - val_accuracy: 0.6298
Epoch 8/10

41/41 [==============================] - 82s 880ms/step - loss: 0.2492 - accuracy: 0.9228 - val_loss: 1.9910 - val_accuracy: 0.6007
Epoch 2/10
41/41 [==============================] - 46s 760ms/step - loss: 0.2585 - accuracy: 0.9144 - val_loss: 1.5362 - val_accuracy: 0.6733
Epoch 3/10
41/41 [==============================] - 44s 765ms/step - loss: 0.2258 - accuracy: 0.9209 - val_loss: 3.0452 - val_accuracy: 0.5118
Epoch 4/10
41/41 [==============================] - 46s 763ms/step - loss: 0.1388 - accuracy: 0.9537 - val_loss: 2.2102 - val_accuracy: 0.5753
Epoch 5/10
41/41 [==============================] - 47s 762ms/step - loss: 0.2086 - accuracy: 0.9362 - val_loss: 1.6509 - val_accuracy: 0.6497
Epoch 6/10
41/41 [==============================] - 44s 753ms/step - loss: 0.1366 - accuracy: 0.9580 - val_loss: 2.3810 - val_accuracy: 0.5445
Epoch 7/10
41/41 [==============================] - 42s 744ms/step - loss: 0.2244 - accuracy: 0.9337 - val_loss: 2.3801 - val_accuracy: 0.5989
Epoch 8/10

41/41 [==============================] - 78s 874ms/step - loss: 1.6751 - accuracy: 0.2735 - val_loss: 2.8595 - val_accuracy: 0.2668
Epoch 2/10
41/41 [==============================] - 45s 765ms/step - loss: 1.5953 - accuracy: 0.3367 - val_loss: 2.3068 - val_accuracy: 0.2668
Epoch 3/10
41/41 [==============================] - 47s 755ms/step - loss: 1.5582 - accuracy: 0.3937 - val_loss: 1.9514 - val_accuracy: 0.2668
Epoch 4/10
41/41 [==============================] - 45s 757ms/step - loss: 1.5252 - accuracy: 0.4294 - val_loss: 1.7501 - val_accuracy: 0.2668
Epoch 5/10
41/41 [==============================] - 47s 754ms/step - loss: 1.4947 - accuracy: 0.4343 - val_loss: 1.6474 - val_accuracy: 0.2668
Epoch 6/10
41/41 [==============================] - 39s 757ms/step - loss: 1.4575 - accuracy: 0.4570 - val_loss: 1.6124 - val_accuracy: 0.2668
Epoch 7/10
41/41 [==============================] - 45s 751ms/step - loss: 1.4254 - accuracy: 0.4946 - val_loss: 1.5455 - val_accuracy: 0.2777
Epoch 8/10

41/41 [==============================] - 81s 862ms/step - loss: 1.1764 - accuracy: 0.5292 - val_loss: 4.7785 - val_accuracy: 0.4011
Epoch 2/10
41/41 [==============================] - 43s 742ms/step - loss: 0.8423 - accuracy: 0.6613 - val_loss: 1.7942 - val_accuracy: 0.5118
Epoch 3/10
41/41 [==============================] - 48s 756ms/step - loss: 0.6695 - accuracy: 0.7291 - val_loss: 3.5444 - val_accuracy: 0.4773
Epoch 4/10
41/41 [==============================] - 46s 746ms/step - loss: 0.5278 - accuracy: 0.8037 - val_loss: 1.9098 - val_accuracy: 0.5989
Epoch 5/10
41/41 [==============================] - 44s 749ms/step - loss: 0.4292 - accuracy: 0.8467 - val_loss: 2.3211 - val_accuracy: 0.5408
Epoch 6/10
41/41 [==============================] - 44s 754ms/step - loss: 0.2868 - accuracy: 0.8968 - val_loss: 1.5575 - val_accuracy: 0.6225
Epoch 7/10
41/41 [==============================] - 43s 736ms/step - loss: 0.2703 - accuracy: 0.9011 - val_loss: 1.8320 - val_accuracy: 0.6207
Epoch 8/10

In [12]:
prune_model(model_path='beans_models/MobileNetV2_beans_model.h5', 
            ds_train=beans_datasets[0], 
            ds_validation=beans_datasets[1],
            batch_size=BATCH_SIZE, 
            pruning_epochs=PRUNING_EPOCHS,
            sparsity=0.5)
prune_model(model_path='beans_models/MobileNetV2_beans_model.h5', 
            ds_train=beans_datasets[0], 
            ds_validation=beans_datasets[1],
            batch_size=BATCH_SIZE, 
            pruning_epochs=PRUNING_EPOCHS,
            sparsity=0.75)
prune_model(model_path='beans_models/MobileNetV2_beans_model.h5', 
            ds_train=beans_datasets[0], 
            ds_validation=beans_datasets[1],
            batch_size=BATCH_SIZE, 
            pruning_epochs=PRUNING_EPOCHS,
            sparsity=0.9)

prune_model(model_path='beans_models/EfficentNetB0_beans_model.h5', 
            ds_train=beans_datasets[0], 
            ds_validation=beans_datasets[1], 
            batch_size=BATCH_SIZE, 
            pruning_epochs=PRUNING_EPOCHS,
            sparsity=0.5)
prune_model(model_path='beans_models/EfficentNetB0_beans_model.h5', 
            ds_train=beans_datasets[0], 
            ds_validation=beans_datasets[1], 
            batch_size=BATCH_SIZE, 
            pruning_epochs=PRUNING_EPOCHS,
            sparsity=0.75)
prune_model(model_path='beans_models/EfficentNetB0_beans_model.h5', 
            ds_train=beans_datasets[0], 
            ds_validation=beans_datasets[1], 
            batch_size=BATCH_SIZE, 
            pruning_epochs=PRUNING_EPOCHS,
            sparsity=0.9)

Epoch 1/10
 6/17 [=========>....................] - ETA: 23s - loss: 0.7109 - accuracy: 0.8349WARNING:tensorflow:Callback method `on_train_batch_end` is slow compared to the batch time (batch time: 0.4215s vs `on_train_batch_end` time: 1.4311s). Check your callbacks.


17/17 [==============================] - 58s 2s/step - loss: 0.6173 - accuracy: 0.8374 - val_loss: 2.2560 - val_accuracy: 0.4962
Epoch 2/10
17/17 [==============================] - 33s 2s/step - loss: 0.1355 - accuracy: 0.9450 - val_loss: 2.7642 - val_accuracy: 0.5414
Epoch 3/10
17/17 [==============================] - 33s 2s/step - loss: 0.0654 - accuracy: 0.9780 - val_loss: 3.6966 - val_accuracy: 0.4361
Epoch 4/10
17/17 [==============================] - 33s 2s/step - loss: 0.0332 - accuracy: 0.9900 - val_loss: 3.5461 - val_accuracy: 0.4812
Epoch 5/10
17/17 [==============================] - 33s 2s/step - loss: 0.0177 - accuracy: 0.9949 - val_loss: 4.0588 - val_accuracy: 0.4211
Epoch 6/10
17/17 [==============================] - 33s 2s/step - loss: 0.0062 - accuracy: 0.9997 - val_loss: 3.4860 - val_accuracy: 0.5865
Epoch 7/10
17/17 [==============================] - 33s 2s/step - loss: 0.0043 - accuracy: 0.9997 - val_loss: 3.0434 - val_accuracy: 0.5263
Epoch 8/10
17/17 [=============

17/17 [==============================] - 56s 2s/step - loss: 0.1104 - accuracy: 0.9569 - val_loss: 8.9401 - val_accuracy: 0.4737
Epoch 2/10
17/17 [==============================] - 33s 2s/step - loss: 0.1979 - accuracy: 0.9341 - val_loss: 10.1109 - val_accuracy: 0.3985
Epoch 3/10
17/17 [==============================] - 33s 2s/step - loss: 0.0802 - accuracy: 0.9695 - val_loss: 15.9307 - val_accuracy: 0.3985
Epoch 4/10
17/17 [==============================] - 33s 2s/step - loss: 0.0927 - accuracy: 0.9715 - val_loss: 25.5463 - val_accuracy: 0.4211
Epoch 5/10
17/17 [==============================] - 33s 2s/step - loss: 0.1106 - accuracy: 0.9579 - val_loss: 6.4101 - val_accuracy: 0.5113
Epoch 6/10
17/17 [==============================] - 33s 2s/step - loss: 0.1690 - accuracy: 0.9528 - val_loss: 17.1670 - val_accuracy: 0.3609
Epoch 7/10
17/17 [==============================] - 33s 2s/step - loss: 0.0573 - accuracy: 0.9834 - val_loss: 11.3339 - val_accuracy: 0.5038
Epoch 8/10
17/17 [========

17/17 [==============================] - 54s 2s/step - loss: 1.5089 - accuracy: 0.5947 - val_loss: 1.2537 - val_accuracy: 0.3308
Epoch 2/10
17/17 [==============================] - 33s 2s/step - loss: 0.4702 - accuracy: 0.8179 - val_loss: 1.2996 - val_accuracy: 0.3308
Epoch 3/10
17/17 [==============================] - 33s 2s/step - loss: 0.2334 - accuracy: 0.9313 - val_loss: 1.4627 - val_accuracy: 0.3308
Epoch 4/10
17/17 [==============================] - 33s 2s/step - loss: 0.1495 - accuracy: 0.9627 - val_loss: 1.6062 - val_accuracy: 0.3308
Epoch 5/10
17/17 [==============================] - 33s 2s/step - loss: 0.0921 - accuracy: 0.9761 - val_loss: 1.7679 - val_accuracy: 0.3308
Epoch 6/10
17/17 [==============================] - 33s 2s/step - loss: 0.0511 - accuracy: 0.9932 - val_loss: 1.8862 - val_accuracy: 0.3308
Epoch 7/10
17/17 [==============================] - 32s 2s/step - loss: 0.0446 - accuracy: 0.9915 - val_loss: 1.9919 - val_accuracy: 0.3308
Epoch 8/10
17/17 [=============

17/17 [==============================] - 55s 2s/step - loss: 0.1570 - accuracy: 0.9459 - val_loss: 1.8077 - val_accuracy: 0.3308
Epoch 2/10
17/17 [==============================] - 33s 2s/step - loss: 0.3782 - accuracy: 0.8765 - val_loss: 3.6830 - val_accuracy: 0.3308
Epoch 3/10
17/17 [==============================] - 33s 2s/step - loss: 0.1604 - accuracy: 0.9366 - val_loss: 3.1757 - val_accuracy: 0.3308
Epoch 4/10
17/17 [==============================] - 33s 2s/step - loss: 0.1130 - accuracy: 0.9609 - val_loss: 3.1177 - val_accuracy: 0.3308
Epoch 5/10
17/17 [==============================] - 32s 2s/step - loss: 0.1461 - accuracy: 0.9433 - val_loss: 3.3786 - val_accuracy: 0.3308
Epoch 6/10
17/17 [==============================] - 32s 2s/step - loss: 0.1281 - accuracy: 0.9591 - val_loss: 3.3704 - val_accuracy: 0.3308
Epoch 7/10
17/17 [==============================] - 33s 2s/step - loss: 0.1252 - accuracy: 0.9551 - val_loss: 3.2827 - val_accuracy: 0.3308
Epoch 8/10
17/17 [=============

17/17 [==============================] - 52s 2s/step - loss: 1.3020 - accuracy: 0.4931 - val_loss: 1.1042 - val_accuracy: 0.3383
Epoch 2/10
17/17 [==============================] - 33s 2s/step - loss: 0.6675 - accuracy: 0.7117 - val_loss: 1.1023 - val_accuracy: 0.3383
Epoch 3/10
17/17 [==============================] - 33s 2s/step - loss: 0.4820 - accuracy: 0.8027 - val_loss: 1.1055 - val_accuracy: 0.3383
Epoch 4/10
17/17 [==============================] - 33s 2s/step - loss: 0.3557 - accuracy: 0.8892 - val_loss: 1.1102 - val_accuracy: 0.3383
Epoch 5/10
17/17 [==============================] - 33s 2s/step - loss: 0.2281 - accuracy: 0.9470 - val_loss: 1.1101 - val_accuracy: 0.3383
Epoch 6/10
17/17 [==============================] - 33s 2s/step - loss: 0.1793 - accuracy: 0.9487 - val_loss: 1.1108 - val_accuracy: 0.3383
Epoch 7/10
17/17 [==============================] - 33s 2s/step - loss: 0.1624 - accuracy: 0.9655 - val_loss: 1.1069 - val_accuracy: 0.3383
Epoch 8/10
17/17 [=============

17/17 [==============================] - 54s 2s/step - loss: 0.5742 - accuracy: 0.7934 - val_loss: 1.1280 - val_accuracy: 0.3383
Epoch 2/10
17/17 [==============================] - 32s 2s/step - loss: 0.5205 - accuracy: 0.8057 - val_loss: 1.0991 - val_accuracy: 0.3308
Epoch 3/10
17/17 [==============================] - 33s 2s/step - loss: 0.3294 - accuracy: 0.8848 - val_loss: 1.1504 - val_accuracy: 0.3308
Epoch 4/10
17/17 [==============================] - 33s 2s/step - loss: 0.2145 - accuracy: 0.9261 - val_loss: 1.1187 - val_accuracy: 0.3308
Epoch 5/10
17/17 [==============================] - 33s 2s/step - loss: 0.2005 - accuracy: 0.9215 - val_loss: 1.1686 - val_accuracy: 0.3308
Epoch 6/10
17/17 [==============================] - 33s 2s/step - loss: 0.2014 - accuracy: 0.9217 - val_loss: 1.1359 - val_accuracy: 0.3308
Epoch 7/10
17/17 [==============================] - 33s 2s/step - loss: 0.2323 - accuracy: 0.9159 - val_loss: 1.1177 - val_accuracy: 0.3308
Epoch 8/10
17/17 [=============

ResourceExhaustedError:  OOM when allocating tensor with shape[64,480,32,32] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node model_1/prune_low_magnitude_block4c_dwconv/depthwise (defined at /home/macenpav/.conda/envs/ml/lib/python3.9/site-packages/tensorflow_model_optimization/python/core/sparsity/keras/pruning_wrapper.py:270) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.
 [Op:__inference_train_function_2364702]

Function call stack:
train_function


## 2. Weight cluster base models

In [13]:
weight_cluster_model(model_path='flowers_models/MobileNetV2_flowers_model.h5', 
                     ds_train=flowers_datasets[0], 
                     ds_validation=flowers_datasets[1], 
                     batch_size=BATCH_SIZE, 
                     epochs=PRUNING_EPOCHS, 
                     number_of_clusters=32)
weight_cluster_model(model_path='flowers_models/MobileNetV2_flowers_model.h5', 
                     ds_train=flowers_datasets[0], 
                     ds_validation=flowers_datasets[1], 
                     batch_size=BATCH_SIZE, 
                     epochs=PRUNING_EPOCHS, 
                     number_of_clusters=128)

weight_cluster_model(model_path='flowers_models/EfficentNetB0_flowers_model.h5', 
                     ds_train=flowers_datasets[0], 
                     ds_validation=flowers_datasets[1], 
                     batch_size=BATCH_SIZE, 
                     epochs=PRUNING_EPOCHS, 
                     number_of_clusters=32)
weight_cluster_model(model_path='flowers_models/EfficentNetB0_flowers_model.h5', 
                     ds_train=flowers_datasets[0], 
                     ds_validation=flowers_datasets[1], 
                     batch_size=BATCH_SIZE, 
                     epochs=PRUNING_EPOCHS, 
                     number_of_clusters=128)

Epoch 1/10
41/41 [==============================] - 44s 559ms/step - loss: 0.2976 - accuracy: 0.9421 - val_loss: 1.2797 - val_accuracy: 0.6915
Epoch 2/10
41/41 [==============================] - 36s 532ms/step - loss: 0.0087 - accuracy: 0.9981 - val_loss: 1.8917 - val_accuracy: 0.6969
Epoch 3/10
41/41 [==============================] - 33s 538ms/step - loss: 0.0100 - accuracy: 0.9965 - val_loss: 1.8013 - val_accuracy: 0.7078
Epoch 4/10
41/41 [==============================] - 36s 541ms/step - loss: 0.0030 - accuracy: 0.9990 - val_loss: 1.8918 - val_accuracy: 0.6897
Epoch 5/10
41/41 [==============================] - 36s 523ms/step - loss: 8.3424e-04 - accuracy: 0.9998 - val_loss: 1.8506 - val_accuracy: 0.7169
Epoch 6/10
41/41 [==============================] - 36s 532ms/step - loss: 5.2407e-04 - accuracy: 0.9998 - val_loss: 1.9298 - val_accuracy: 0.7169
Epoch 7/10
41/41 [==============================] - 35s 533ms/step - loss: 9.5425e-04 - accuracy: 0.9991 - val_loss: 1.8869 - val_accu

In [14]:
weight_cluster_model(model_path='beans_models/MobileNetV2_beans_model.h5', 
                     ds_train=beans_datasets[0], 
                     ds_validation=beans_datasets[1], 
                     batch_size=BATCH_SIZE, 
                     epochs=PRUNING_EPOCHS, 
                     number_of_clusters=32)
weight_cluster_model(model_path='beans_models/MobileNetV2_beans_model.h5', 
                     ds_train=beans_datasets[0], 
                     ds_validation=beans_datasets[1], 
                     batch_size=BATCH_SIZE, 
                     epochs=PRUNING_EPOCHS, 
                     number_of_clusters=128)

weight_cluster_model(model_path='beans_models/EfficentNetB0_beans_model.h5', 
                     ds_train=beans_datasets[0], 
                     ds_validation=beans_datasets[1], 
                     batch_size=BATCH_SIZE, 
                     epochs=PRUNING_EPOCHS, 
                     number_of_clusters=32)
weight_cluster_model(model_path='beans_models/EfficentNetB0_beans_model.h5', 
                     ds_train=beans_datasets[0], 
                     ds_validation=beans_datasets[1], 
                     batch_size=BATCH_SIZE, 
                     epochs=PRUNING_EPOCHS, 
                     number_of_clusters=128)

Epoch 1/10
17/17 [==============================] - 40s 2s/step - loss: 6.5026 - accuracy: 0.7062 - val_loss: 1.0236 - val_accuracy: 0.5188
Epoch 2/10
17/17 [==============================] - 35s 2s/step - loss: 0.9987 - accuracy: 0.4653 - val_loss: 0.9118 - val_accuracy: 0.5489
Epoch 3/10
17/17 [==============================] - 35s 2s/step - loss: 0.7917 - accuracy: 0.6053 - val_loss: 0.7014 - val_accuracy: 0.6692
Epoch 4/10
17/17 [==============================] - 35s 2s/step - loss: 0.5710 - accuracy: 0.7508 - val_loss: 0.9609 - val_accuracy: 0.6466
Epoch 5/10
17/17 [==============================] - 34s 2s/step - loss: 0.5487 - accuracy: 0.7576 - val_loss: 0.8500 - val_accuracy: 0.6617
Epoch 6/10
17/17 [==============================] - 35s 2s/step - loss: 0.4149 - accuracy: 0.8378 - val_loss: 1.1032 - val_accuracy: 0.6617
Epoch 7/10
17/17 [==============================] - 35s 2s/step - loss: 0.3102 - accuracy: 0.8811 - val_loss: 0.9017 - val_accuracy: 0.6917
Epoch 8/10
17/17 [==

ResourceExhaustedError:  OOM when allocating tensor with shape[64,480,32,32] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node model_1/cluster_block4c_dwconv/depthwise (defined at /home/macenpav/.conda/envs/ml/lib/python3.9/site-packages/tensorflow_model_optimization/python/core/clustering/keras/cluster_wrapper.py:265) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.
 [Op:__inference_train_function_2563251]

Function call stack:
train_function


## 3. prune weight clustered models

In [15]:
prune_model(model_path='flowers_models_optimized/MobileNetV2_flowers_model_KMeansPlusPlus32.h5', 
            ds_train=flowers_datasets[0], 
            ds_validation=flowers_datasets[1],
            batch_size=BATCH_SIZE, 
            pruning_epochs=PRUNING_EPOCHS,
            sparsity=0.5)
prune_model(model_path='flowers_models_optimized/MobileNetV2_flowers_model_KMeansPlusPlus32.h5', 
            ds_train=flowers_datasets[0], 
            ds_validation=flowers_datasets[1],
            batch_size=BATCH_SIZE, 
            pruning_epochs=PRUNING_EPOCHS,
            sparsity=0.75)
prune_model(model_path='flowers_models_optimized/MobileNetV2_flowers_model_KMeansPlusPlus32.h5', 
            ds_train=flowers_datasets[0], 
            ds_validation=flowers_datasets[1],
            batch_size=BATCH_SIZE, 
            pruning_epochs=PRUNING_EPOCHS,
            sparsity=0.9)

prune_model(model_path='flowers_models_optimized/MobileNetV2_flowers_model_KMeansPlusPlus128.h5', 
            ds_train=flowers_datasets[0], 
            ds_validation=flowers_datasets[1],
            batch_size=BATCH_SIZE, 
            pruning_epochs=PRUNING_EPOCHS,
            sparsity=0.5)
prune_model(model_path='flowers_models_optimized/MobileNetV2_flowers_model_KMeansPlusPlus128.h5', 
            ds_train=flowers_datasets[0], 
            ds_validation=flowers_datasets[1],
            batch_size=BATCH_SIZE, 
            pruning_epochs=PRUNING_EPOCHS,
            sparsity=0.75)
prune_model(model_path='flowers_models_optimized/MobileNetV2_flowers_model_KMeansPlusPlus128.h5', 
            ds_train=flowers_datasets[0], 
            ds_validation=flowers_datasets[1],
            batch_size=BATCH_SIZE, 
            pruning_epochs=PRUNING_EPOCHS,
            sparsity=0.9)

# ---------- EfficentNet ---------

prune_model(model_path='flowers_models_optimized/EfficentNetB0_flowers_model_KMeansPlusPlus32.h5', 
            ds_train=flowers_datasets[0], 
            ds_validation=flowers_datasets[1],
            batch_size=BATCH_SIZE, 
            pruning_epochs=PRUNING_EPOCHS,
            sparsity=0.5)
prune_model(model_path='flowers_models_optimized/EfficentNetB0_flowers_model_KMeansPlusPlus32.h5', 
            ds_train=flowers_datasets[0], 
            ds_validation=flowers_datasets[1],
            batch_size=BATCH_SIZE, 
            pruning_epochs=PRUNING_EPOCHS,
            sparsity=0.75)
prune_model(model_path='flowers_models_optimized/EfficentNetB0_flowers_model_KMeansPlusPlus32.h5', 
            ds_train=flowers_datasets[0], 
            ds_validation=flowers_datasets[1],
            batch_size=BATCH_SIZE, 
            pruning_epochs=PRUNING_EPOCHS,
            sparsity=0.9)

prune_model(model_path='flowers_models_optimized/EfficentNetB0_flowers_model_KMeansPlusPlus128.h5', 
            ds_train=flowers_datasets[0], 
            ds_validation=flowers_datasets[1],
            batch_size=BATCH_SIZE, 
            pruning_epochs=PRUNING_EPOCHS,
            sparsity=0.5)
prune_model(model_path='flowers_models_optimized/EfficentNetB0_flowers_model_KMeansPlusPlus128.h5', 
            ds_train=flowers_datasets[0], 
            ds_validation=flowers_datasets[1],
            batch_size=BATCH_SIZE, 
            pruning_epochs=PRUNING_EPOCHS,
            sparsity=0.75)
prune_model(model_path='flowers_models_optimized/EfficentNetB0_flowers_model_KMeansPlusPlus128.h5', 
            ds_train=flowers_datasets[0], 
            ds_validation=flowers_datasets[1],
            batch_size=BATCH_SIZE, 
            pruning_epochs=PRUNING_EPOCHS,
            sparsity=0.9)

Epoch 1/10
 6/41 [===>..........................] - ETA: 1:03 - loss: 0.2037 - accuracy: 0.9411WARNING:tensorflow:Callback method `on_train_batch_end` is slow compared to the batch time (batch time: 0.1570s vs `on_train_batch_end` time: 1.3921s). Check your callbacks.


41/41 [==============================] - 63s 729ms/step - loss: 0.2918 - accuracy: 0.9147 - val_loss: 6.7682 - val_accuracy: 0.2995
Epoch 2/10
41/41 [==============================] - 37s 521ms/step - loss: 0.1070 - accuracy: 0.9646 - val_loss: 5.7078 - val_accuracy: 0.3103
Epoch 3/10
41/41 [==============================] - 35s 520ms/step - loss: 0.0534 - accuracy: 0.9814 - val_loss: 5.7805 - val_accuracy: 0.2958
Epoch 4/10
41/41 [==============================] - 36s 527ms/step - loss: 0.0520 - accuracy: 0.9814 - val_loss: 4.6590 - val_accuracy: 0.3666
Epoch 5/10
41/41 [==============================] - 35s 555ms/step - loss: 0.0327 - accuracy: 0.9895 - val_loss: 4.1447 - val_accuracy: 0.4755
Epoch 6/10
41/41 [==============================] - 37s 551ms/step - loss: 0.0594 - accuracy: 0.9847 - val_loss: 2.7174 - val_accuracy: 0.5735
Epoch 7/10
41/41 [==============================] - 36s 532ms/step - loss: 0.0286 - accuracy: 0.9886 - val_loss: 2.7004 - val_accuracy: 0.5590
Epoch 8/10

41/41 [==============================] - 56s 600ms/step - loss: 0.1022 - accuracy: 0.9689 - val_loss: 6.2461 - val_accuracy: 0.5045
Epoch 2/10
41/41 [==============================] - 34s 515ms/step - loss: 0.2067 - accuracy: 0.9333 - val_loss: 7.6332 - val_accuracy: 0.4428
Epoch 3/10
41/41 [==============================] - 34s 519ms/step - loss: 0.2349 - accuracy: 0.9161 - val_loss: 4.6868 - val_accuracy: 0.5681
Epoch 4/10
41/41 [==============================] - 34s 519ms/step - loss: 0.0876 - accuracy: 0.9679 - val_loss: 6.5540 - val_accuracy: 0.4791
Epoch 5/10
41/41 [==============================] - 33s 523ms/step - loss: 0.0216 - accuracy: 0.9927 - val_loss: 6.4091 - val_accuracy: 0.5191
Epoch 6/10
41/41 [==============================] - 32s 541ms/step - loss: 0.0695 - accuracy: 0.9773 - val_loss: 6.4849 - val_accuracy: 0.5209
Epoch 7/10
41/41 [==============================] - 38s 535ms/step - loss: 0.0281 - accuracy: 0.9935 - val_loss: 6.6971 - val_accuracy: 0.5699
Epoch 8/10

Epoch 1/10
 6/41 [===>..........................] - ETA: 30s - loss: 2.3102 - accuracy: 0.5066WARNING:tensorflow:Callback method `on_train_batch_end` is slow compared to the batch time (batch time: 0.1482s vs `on_train_batch_end` time: 0.5934s). Check your callbacks.


41/41 [==============================] - 56s 616ms/step - loss: 1.5135 - accuracy: 0.6039 - val_loss: 4.7603 - val_accuracy: 0.2668
Epoch 2/10
41/41 [==============================] - 35s 536ms/step - loss: 0.4117 - accuracy: 0.8448 - val_loss: 5.0208 - val_accuracy: 0.2668
Epoch 3/10
41/41 [==============================] - 38s 522ms/step - loss: 0.2011 - accuracy: 0.9379 - val_loss: 5.3697 - val_accuracy: 0.2668
Epoch 4/10
41/41 [==============================] - 32s 516ms/step - loss: 0.0889 - accuracy: 0.9809 - val_loss: 5.9941 - val_accuracy: 0.2668
Epoch 5/10
41/41 [==============================] - 37s 544ms/step - loss: 0.0868 - accuracy: 0.9751 - val_loss: 6.5041 - val_accuracy: 0.2668
Epoch 6/10
41/41 [==============================] - 38s 533ms/step - loss: 0.0616 - accuracy: 0.9825 - val_loss: 6.4043 - val_accuracy: 0.2668
Epoch 7/10
41/41 [==============================] - 36s 529ms/step - loss: 0.0523 - accuracy: 0.9853 - val_loss: 6.8116 - val_accuracy: 0.2668
Epoch 8/10

...

41/41 [==============================] - 86s 904ms/step - loss: 1.2873 - accuracy: 0.4884 - val_loss: 1.8194 - val_accuracy: 0.2668
Epoch 2/10
41/41 [==============================] - 49s 782ms/step - loss: 0.8221 - accuracy: 0.7076 - val_loss: 1.9122 - val_accuracy: 0.2668
Epoch 3/10
41/41 [==============================] - 48s 757ms/step - loss: 0.5850 - accuracy: 0.8096 - val_loss: 1.8176 - val_accuracy: 0.2922
Epoch 4/10
41/41 [==============================] - 43s 767ms/step - loss: 0.3702 - accuracy: 0.8996 - val_loss: 1.3227 - val_accuracy: 0.4846
Epoch 5/10
41/41 [==============================] - 45s 739ms/step - loss: 0.2879 - accuracy: 0.9202 - val_loss: 1.2966 - val_accuracy: 0.5354
Epoch 6/10
41/41 [==============================] - 42s 760ms/step - loss: 0.2363 - accuracy: 0.9333 - val_loss: 1.0902 - val_accuracy: 0.5808
Epoch 7/10
41/41 [==============================] - 47s 760ms/step - loss: 0.1639 - accuracy: 0.9573 - val_loss: 1.2420 - val_accuracy: 0.5808
Epoch 8/10

41/41 [==============================] - 82s 873ms/step - loss: 0.2153 - accuracy: 0.9301 - val_loss: 2.4173 - val_accuracy: 0.5644
Epoch 2/10
41/41 [==============================] - 47s 754ms/step - loss: 0.3196 - accuracy: 0.8997 - val_loss: 1.7740 - val_accuracy: 0.6352
Epoch 3/10
41/41 [==============================] - 44s 757ms/step - loss: 0.2157 - accuracy: 0.9305 - val_loss: 1.6498 - val_accuracy: 0.6388
Epoch 4/10
41/41 [==============================] - 45s 758ms/step - loss: 0.1482 - accuracy: 0.9463 - val_loss: 1.6274 - val_accuracy: 0.6225
Epoch 5/10
41/41 [==============================] - 46s 765ms/step - loss: 0.2197 - accuracy: 0.9281 - val_loss: 1.6120 - val_accuracy: 0.6134
Epoch 6/10
41/41 [==============================] - 43s 758ms/step - loss: 0.1910 - accuracy: 0.9358 - val_loss: 1.7658 - val_accuracy: 0.6298
Epoch 7/10
41/41 [==============================] - 42s 742ms/step - loss: 0.1251 - accuracy: 0.9541 - val_loss: 1.8327 - val_accuracy: 0.6588
Epoch 8/10

Epoch 1/10
 6/41 [===>..........................] - ETA: 46s - loss: 1.7451 - accuracy: 0.2647WARNING:tensorflow:Callback method `on_train_batch_end` is slow compared to the batch time (batch time: 0.2458s vs `on_train_batch_end` time: 0.8952s). Check your callbacks.


41/41 [==============================] - 80s 865ms/step - loss: 1.6654 - accuracy: 0.2989 - val_loss: 2.7387 - val_accuracy: 0.2668
Epoch 2/10
41/41 [==============================] - 45s 773ms/step - loss: 1.5503 - accuracy: 0.3850 - val_loss: 2.1709 - val_accuracy: 0.2668
Epoch 3/10
41/41 [==============================] - 45s 758ms/step - loss: 1.5051 - accuracy: 0.4404 - val_loss: 1.8549 - val_accuracy: 0.2668
Epoch 4/10
41/41 [==============================] - 46s 759ms/step - loss: 1.4487 - accuracy: 0.4825 - val_loss: 1.6975 - val_accuracy: 0.2668
Epoch 5/10
41/41 [==============================] - 43s 760ms/step - loss: 1.4075 - accuracy: 0.5006 - val_loss: 1.6277 - val_accuracy: 0.2668
Epoch 6/10
41/41 [==============================] - 47s 757ms/step - loss: 1.3677 - accuracy: 0.5190 - val_loss: 1.6101 - val_accuracy: 0.2668
Epoch 7/10
41/41 [==============================] - 45s 761ms/step - loss: 1.3283 - accuracy: 0.5158 - val_loss: 1.5590 - val_accuracy: 0.3049
Epoch 8/10

41/41 [==============================] - 80s 875ms/step - loss: 1.1534 - accuracy: 0.5191 - val_loss: 3.2734 - val_accuracy: 0.3557
Epoch 2/10
41/41 [==============================] - 46s 752ms/step - loss: 0.8468 - accuracy: 0.6466 - val_loss: 3.8845 - val_accuracy: 0.4083
Epoch 3/10
41/41 [==============================] - 44s 748ms/step - loss: 0.7729 - accuracy: 0.6852 - val_loss: 2.2636 - val_accuracy: 0.5245
Epoch 4/10
41/41 [==============================] - 47s 757ms/step - loss: 0.5431 - accuracy: 0.7885 - val_loss: 2.2428 - val_accuracy: 0.5735
Epoch 5/10
41/41 [==============================] - 47s 757ms/step - loss: 0.3797 - accuracy: 0.8592 - val_loss: 2.4718 - val_accuracy: 0.5299
Epoch 6/10
41/41 [==============================] - 46s 752ms/step - loss: 0.2857 - accuracy: 0.8950 - val_loss: 3.1802 - val_accuracy: 0.5445
Epoch 7/10
41/41 [==============================] - 45s 767ms/step - loss: 0.3924 - accuracy: 0.8535 - val_loss: 2.2447 - val_accuracy: 0.5989
Epoch 8/10

In [16]:
prune_model(model_path='beans_models_optimized/MobileNetV2_beans_model_KMeansPlusPlus32.h5', 
            ds_train=beans_datasets[0], 
            ds_validation=beans_datasets[1],
            batch_size=BATCH_SIZE, 
            pruning_epochs=PRUNING_EPOCHS,
            sparsity=0.5)
prune_model(model_path='beans_models_optimized/MobileNetV2_beans_model_KMeansPlusPlus32.h5', 
            ds_train=beans_datasets[0], 
            ds_validation=beans_datasets[1],
            batch_size=BATCH_SIZE, 
            pruning_epochs=PRUNING_EPOCHS,
            sparsity=0.75)
prune_model(model_path='beans_models_optimized/MobileNetV2_beans_model_KMeansPlusPlus32.h5', 
            ds_train=beans_datasets[0], 
            ds_validation=beans_datasets[1],
            batch_size=BATCH_SIZE, 
            pruning_epochs=PRUNING_EPOCHS,
            sparsity=0.9)

prune_model(model_path='beans_models_optimized/MobileNetV2_beans_model_KMeansPlusPlus128.h5', 
            ds_train=beans_datasets[0], 
            ds_validation=beans_datasets[1],
            batch_size=BATCH_SIZE, 
            pruning_epochs=PRUNING_EPOCHS,
            sparsity=0.5)
prune_model(model_path='beans_models_optimized/MobileNetV2_beans_model_KMeansPlusPlus128.h5', 
            ds_train=beans_datasets[0], 
            ds_validation=beans_datasets[1],
            batch_size=BATCH_SIZE, 
            pruning_epochs=PRUNING_EPOCHS,
            sparsity=0.75)
prune_model(model_path='beans_models_optimized/MobileNetV2_beans_model_KMeansPlusPlus128.h5', 
            ds_train=beans_datasets[0], 
            ds_validation=beans_datasets[1],
            batch_size=BATCH_SIZE, 
            pruning_epochs=PRUNING_EPOCHS,
            sparsity=0.9)

# ---------- EfficentNet ---------

prune_model(model_path='beans_models_optimized/EfficentNetB0_beans_model_KMeansPlusPlus32.h5', 
            ds_train=beans_datasets[0], 
            ds_validation=beans_datasets[1],
            batch_size=BATCH_SIZE, 
            pruning_epochs=PRUNING_EPOCHS,
            sparsity=0.5)
prune_model(model_path='beans_models_optimized/EfficentNetB0_beans_model_KMeansPlusPlus32.h5', 
            ds_train=beans_datasets[0], 
            ds_validation=beans_datasets[1],
            batch_size=BATCH_SIZE, 
            pruning_epochs=PRUNING_EPOCHS,
            sparsity=0.75)
prune_model(model_path='beans_models_optimized/EfficentNetB0_beans_model_KMeansPlusPlus32.h5', 
            ds_train=beans_datasets[0], 
            ds_validation=beans_datasets[1],
            batch_size=BATCH_SIZE, 
            pruning_epochs=PRUNING_EPOCHS,
            sparsity=0.9)

prune_model(model_path='beans_models_optimized/EfficentNetB0_beans_model_KMeansPlusPlus128.h5', 
            ds_train=beans_datasets[0], 
            ds_validation=beans_datasets[1],
            batch_size=BATCH_SIZE, 
            pruning_epochs=PRUNING_EPOCHS,
            sparsity=0.5)
prune_model(model_path='beans_models_optimized/EfficentNetB0_beans_model_KMeansPlusPlus128.h5', 
            ds_train=beans_datasets[0], 
            ds_validation=beans_datasets[1],
            batch_size=BATCH_SIZE, 
            pruning_epochs=PRUNING_EPOCHS,
            sparsity=0.75)
prune_model(model_path='beans_models_optimized/EfficentNetB0_beans_model_KMeansPlusPlus128.h5', 
            ds_train=beans_datasets[0], 
            ds_validation=beans_datasets[1],
            batch_size=BATCH_SIZE, 
            pruning_epochs=PRUNING_EPOCHS,
            sparsity=0.9)

Epoch 1/10
 6/17 [=========>....................] - ETA: 24s - loss: 0.5572 - accuracy: 0.8424WARNING:tensorflow:Callback method `on_train_batch_end` is slow compared to the batch time (batch time: 0.4165s vs `on_train_batch_end` time: 1.6982s). Check your callbacks.


17/17 [==============================] - 57s 2s/step - loss: 0.6024 - accuracy: 0.8292 - val_loss: 1.5216 - val_accuracy: 0.3835
Epoch 2/10
17/17 [==============================] - 32s 2s/step - loss: 0.2121 - accuracy: 0.9311 - val_loss: 3.6439 - val_accuracy: 0.3835
Epoch 3/10
17/17 [==============================] - 33s 2s/step - loss: 0.1485 - accuracy: 0.9434 - val_loss: 1.2939 - val_accuracy: 0.4737
Epoch 4/10
17/17 [==============================] - 33s 2s/step - loss: 0.0854 - accuracy: 0.9729 - val_loss: 2.4903 - val_accuracy: 0.3910
Epoch 5/10
17/17 [==============================] - 33s 2s/step - loss: 0.0522 - accuracy: 0.9851 - val_loss: 2.3708 - val_accuracy: 0.4286
Epoch 6/10
17/17 [==============================] - 33s 2s/step - loss: 0.0417 - accuracy: 0.9893 - val_loss: 3.2931 - val_accuracy: 0.4135
Epoch 7/10
17/17 [==============================] - 32s 2s/step - loss: 0.0324 - accuracy: 0.9922 - val_loss: 2.1208 - val_accuracy: 0.4511
Epoch 8/10
17/17 [=============

17/17 [==============================] - 58s 2s/step - loss: 0.0678 - accuracy: 0.9815 - val_loss: 2.5701 - val_accuracy: 0.5263
Epoch 2/10
17/17 [==============================] - 33s 2s/step - loss: 0.1952 - accuracy: 0.9320 - val_loss: 23.1212 - val_accuracy: 0.3308
Epoch 3/10
17/17 [==============================] - 33s 2s/step - loss: 0.1899 - accuracy: 0.9389 - val_loss: 4.3996 - val_accuracy: 0.5414
Epoch 4/10
17/17 [==============================] - 33s 2s/step - loss: 0.1332 - accuracy: 0.9534 - val_loss: 15.7003 - val_accuracy: 0.5414
Epoch 5/10
17/17 [==============================] - 33s 2s/step - loss: 0.0732 - accuracy: 0.9726 - val_loss: 16.7299 - val_accuracy: 0.4135
Epoch 6/10
17/17 [==============================] - 33s 2s/step - loss: 0.0321 - accuracy: 0.9858 - val_loss: 10.0233 - val_accuracy: 0.4436
Epoch 7/10
17/17 [==============================] - 33s 2s/step - loss: 0.0170 - accuracy: 0.9955 - val_loss: 13.7104 - val_accuracy: 0.4737
Epoch 8/10
17/17 [========

Epoch 1/10
 6/17 [=========>....................] - ETA: 26s - loss: 1.4813 - accuracy: 0.5956WARNING:tensorflow:Callback method `on_train_batch_end` is slow compared to the batch time (batch time: 0.4185s vs `on_train_batch_end` time: 1.7910s). Check your callbacks.


17/17 [==============================] - 59s 2s/step - loss: 1.2926 - accuracy: 0.6172 - val_loss: 1.6039 - val_accuracy: 0.3308
Epoch 2/10
17/17 [==============================] - 32s 2s/step - loss: 0.3628 - accuracy: 0.8666 - val_loss: 1.6120 - val_accuracy: 0.3308
Epoch 3/10
17/17 [==============================] - 33s 2s/step - loss: 0.2315 - accuracy: 0.9360 - val_loss: 1.7327 - val_accuracy: 0.3308
Epoch 4/10
17/17 [==============================] - 33s 2s/step - loss: 0.1236 - accuracy: 0.9652 - val_loss: 1.8903 - val_accuracy: 0.3308
Epoch 5/10
17/17 [==============================] - 33s 2s/step - loss: 0.0639 - accuracy: 0.9863 - val_loss: 2.0397 - val_accuracy: 0.3308
Epoch 6/10
17/17 [==============================] - 33s 2s/step - loss: 0.0328 - accuracy: 0.9965 - val_loss: 2.0226 - val_accuracy: 0.3308
Epoch 7/10
17/17 [==============================] - 33s 2s/step - loss: 0.0460 - accuracy: 0.9893 - val_loss: 2.1514 - val_accuracy: 0.3308
Epoch 8/10
17/17 [=============

...

17/17 [==============================] - 54s 2s/step - loss: 0.2627 - accuracy: 0.9025 - val_loss: 2.0635 - val_accuracy: 0.3308
Epoch 2/10
17/17 [==============================] - 33s 2s/step - loss: 0.3049 - accuracy: 0.8982 - val_loss: 2.3688 - val_accuracy: 0.3759
Epoch 3/10
17/17 [==============================] - 33s 2s/step - loss: 0.1858 - accuracy: 0.9245 - val_loss: 2.3991 - val_accuracy: 0.3308
Epoch 4/10
17/17 [==============================] - 33s 2s/step - loss: 0.0915 - accuracy: 0.9702 - val_loss: 2.3278 - val_accuracy: 0.3308
Epoch 5/10
17/17 [==============================] - 32s 2s/step - loss: 0.1278 - accuracy: 0.9607 - val_loss: 3.6762 - val_accuracy: 0.3308
Epoch 6/10
17/17 [==============================] - 33s 2s/step - loss: 0.0778 - accuracy: 0.9726 - val_loss: 4.6009 - val_accuracy: 0.3308
Epoch 7/10
17/17 [==============================] - 32s 2s/step - loss: 0.0320 - accuracy: 0.9935 - val_loss: 4.3399 - val_accuracy: 0.3308
Epoch 8/10
17/17 [=============

Epoch 1/10
 6/17 [=========>....................] - ETA: 24s - loss: 1.6406 - accuracy: 0.3660WARNING:tensorflow:Callback method `on_train_batch_end` is slow compared to the batch time (batch time: 0.4130s vs `on_train_batch_end` time: 1.6642s). Check your callbacks.


17/17 [==============================] - 53s 2s/step - loss: 1.3877 - accuracy: 0.4288 - val_loss: 1.1055 - val_accuracy: 0.3383
Epoch 2/10
17/17 [==============================] - 33s 2s/step - loss: 0.6365 - accuracy: 0.7508 - val_loss: 1.1088 - val_accuracy: 0.3383
Epoch 3/10
17/17 [==============================] - 33s 2s/step - loss: 0.4780 - accuracy: 0.8362 - val_loss: 1.1159 - val_accuracy: 0.3383
Epoch 4/10
17/17 [==============================] - 32s 2s/step - loss: 0.3623 - accuracy: 0.8765 - val_loss: 1.1287 - val_accuracy: 0.3383
Epoch 5/10
17/17 [==============================] - 33s 2s/step - loss: 0.2439 - accuracy: 0.9539 - val_loss: 1.1395 - val_accuracy: 0.3383
Epoch 6/10
17/17 [==============================] - 32s 2s/step - loss: 0.1793 - accuracy: 0.9617 - val_loss: 1.1403 - val_accuracy: 0.3383
Epoch 7/10
17/17 [==============================] - 33s 2s/step - loss: 0.1381 - accuracy: 0.9699 - val_loss: 1.1377 - val_accuracy: 0.3383
Epoch 8/10
17/17 [=============

17/17 [==============================] - 54s 2s/step - loss: 0.5990 - accuracy: 0.7756 - val_loss: 1.1104 - val_accuracy: 0.3308
Epoch 2/10
17/17 [==============================] - 33s 2s/step - loss: 0.4952 - accuracy: 0.8079 - val_loss: 1.1175 - val_accuracy: 0.3308
Epoch 3/10
17/17 [==============================] - 33s 2s/step - loss: 0.2781 - accuracy: 0.9077 - val_loss: 1.1149 - val_accuracy: 0.3308
Epoch 4/10
17/17 [==============================] - 32s 2s/step - loss: 0.1641 - accuracy: 0.9472 - val_loss: 1.1327 - val_accuracy: 0.3308
Epoch 5/10
17/17 [==============================] - 33s 2s/step - loss: 0.2472 - accuracy: 0.9140 - val_loss: 1.1319 - val_accuracy: 0.3308
Epoch 6/10
17/17 [==============================] - 33s 2s/step - loss: 0.2451 - accuracy: 0.9023 - val_loss: 1.1489 - val_accuracy: 0.3308
Epoch 7/10
17/17 [==============================] - 32s 2s/step - loss: 0.1729 - accuracy: 0.9310 - val_loss: 1.2398 - val_accuracy: 0.3308
Epoch 8/10
17/17 [=============

OSError: SavedModel file does not exist at: beans_models_optimized/EfficentNetB0_beans_model_KMeansPlusPlus32.h5/{saved_model.pbtxt|saved_model.pb}

## 4. weight cluster prunned models

In [ ]:
weight_cluster_model(model_path='flowers_models_optimized/MobileNetV2_flowers_model_ConstantSparsity50.h5', 
                     ds_train=flowers_datasets[0], 
                     ds_validation=flowers_datasets[1], 
                     batch_size=BATCH_SIZE, 
                     epochs=PRUNING_EPOCHS, 
                     number_of_clusters=32)
weight_cluster_model(model_path='flowers_models_optimized/MobileNetV2_flowers_model_ConstantSparsity50.h5', 
                     ds_train=flowers_datasets[0], 
                     ds_validation=flowers_datasets[1], 
                     batch_size=BATCH_SIZE, 
                     epochs=PRUNING_EPOCHS, 
                     number_of_clusters=128)

weight_cluster_model(model_path='flowers_models_optimized/MobileNetV2_flowers_model_PolynomialDecay50.h5', 
                     ds_train=flowers_datasets[0], 
                     ds_validation=flowers_datasets[1], 
                     batch_size=BATCH_SIZE, 
                     epochs=PRUNING_EPOCHS, 
                     number_of_clusters=32)
weight_cluster_model(model_path='flowers_models_optimized/MobileNetV2_flowers_model_PolynomialDecay50.h5', 
                     ds_train=flowers_datasets[0], 
                     ds_validation=flowers_datasets[1], 
                     batch_size=BATCH_SIZE, 
                     epochs=PRUNING_EPOCHS, 
                     number_of_clusters=128)

weight_cluster_model(model_path='flowers_models_optimized/MobileNetV2_flowers_model_ConstantSparsity75.h5', 
                     ds_train=flowers_datasets[0], 
                     ds_validation=flowers_datasets[1], 
                     batch_size=BATCH_SIZE, 
                     epochs=PRUNING_EPOCHS, 
                     number_of_clusters=32)
weight_cluster_model(model_path='flowers_models_optimized/MobileNetV2_flowers_model_ConstantSparsity75.h5', 
                     ds_train=flowers_datasets[0], 
                     ds_validation=flowers_datasets[1], 
                     batch_size=BATCH_SIZE, 
                     epochs=PRUNING_EPOCHS, 
                     number_of_clusters=128)

weight_cluster_model(model_path='flowers_models_optimized/MobileNetV2_flowers_model_PolynomialDecay75.h5', 
                     ds_train=flowers_datasets[0], 
                     ds_validation=flowers_datasets[1], 
                     batch_size=BATCH_SIZE, 
                     epochs=PRUNING_EPOCHS, 
                     number_of_clusters=32)
weight_cluster_model(model_path='flowers_models_optimized/MobileNetV2_flowers_model_PolynomialDecay75.h5', 
                     ds_train=flowers_datasets[0], 
                     ds_validation=flowers_datasets[1], 
                     batch_size=BATCH_SIZE, 
                     epochs=PRUNING_EPOCHS, 
                     number_of_clusters=128)

weight_cluster_model(model_path='flowers_models_optimized/MobileNetV2_flowers_model_ConstantSparsity90.h5', 
                     ds_train=flowers_datasets[0], 
                     ds_validation=flowers_datasets[1], 
                     batch_size=BATCH_SIZE, 
                     epochs=PRUNING_EPOCHS, 
                     number_of_clusters=32)
weight_cluster_model(model_path='flowers_models_optimized/MobileNetV2_flowers_model_ConstantSparsity90.h5', 
                     ds_train=flowers_datasets[0], 
                     ds_validation=flowers_datasets[1], 
                     batch_size=BATCH_SIZE, 
                     epochs=PRUNING_EPOCHS, 
                     number_of_clusters=128)

weight_cluster_model(model_path='flowers_models_optimized/MobileNetV2_flowers_model_PolynomialDecay90.h5', 
                     ds_train=flowers_datasets[0], 
                     ds_validation=flowers_datasets[1], 
                     batch_size=BATCH_SIZE, 
                     epochs=PRUNING_EPOCHS, 
                     number_of_clusters=32)
weight_cluster_model(model_path='flowers_models_optimized/MobileNetV2_flowers_model_PolynomialDecay90.h5', 
                     ds_train=flowers_datasets[0], 
                     ds_validation=flowers_datasets[1], 
                     batch_size=BATCH_SIZE, 
                     epochs=PRUNING_EPOCHS, 
                     number_of_clusters=128)

In [ ]:
weight_cluster_model(model_path='flowers_models_optimized/EfficentNetB0_flowers_model_ConstantSparsity50.h5', 
                     ds_train=flowers_datasets[0], 
                     ds_validation=flowers_datasets[1], 
                     batch_size=BATCH_SIZE, 
                     epochs=PRUNING_EPOCHS, 
                     number_of_clusters=32)
weight_cluster_model(model_path='flowers_models_optimized/EfficentNetB0_flowers_model_ConstantSparsity50.h5', 
                     ds_train=flowers_datasets[0], 
                     ds_validation=flowers_datasets[1], 
                     batch_size=BATCH_SIZE, 
                     epochs=PRUNING_EPOCHS, 
                     number_of_clusters=128)

weight_cluster_model(model_path='flowers_models_optimized/EfficentNetB0_flowers_model_PolynomialDecay50.h5', 
                     ds_train=flowers_datasets[0], 
                     ds_validation=flowers_datasets[1], 
                     batch_size=BATCH_SIZE, 
                     epochs=PRUNING_EPOCHS, 
                     number_of_clusters=32)
weight_cluster_model(model_path='flowers_models_optimized/EfficentNetB0_flowers_model_PolynomialDecay50.h5', 
                     ds_train=flowers_datasets[0], 
                     ds_validation=flowers_datasets[1], 
                     batch_size=BATCH_SIZE, 
                     epochs=PRUNING_EPOCHS, 
                     number_of_clusters=128)

weight_cluster_model(model_path='flowers_models_optimized/EfficentNetB0_flowers_model_ConstantSparsity75.h5', 
                     ds_train=flowers_datasets[0], 
                     ds_validation=flowers_datasets[1], 
                     batch_size=BATCH_SIZE, 
                     epochs=PRUNING_EPOCHS, 
                     number_of_clusters=32)
weight_cluster_model(model_path='flowers_models_optimized/EfficentNetB0_flowers_model_ConstantSparsity75.h5', 
                     ds_train=flowers_datasets[0], 
                     ds_validation=flowers_datasets[1], 
                     batch_size=BATCH_SIZE, 
                     epochs=PRUNING_EPOCHS, 
                     number_of_clusters=128)

weight_cluster_model(model_path='flowers_models_optimized/EfficentNetB0_flowers_model_PolynomialDecay75.h5', 
                     ds_train=flowers_datasets[0], 
                     ds_validation=flowers_datasets[1], 
                     batch_size=BATCH_SIZE, 
                     epochs=PRUNING_EPOCHS, 
                     number_of_clusters=32)
weight_cluster_model(model_path='flowers_models_optimized/EfficentNetB0_flowers_model_PolynomialDecay75.h5', 
                     ds_train=flowers_datasets[0], 
                     ds_validation=flowers_datasets[1], 
                     batch_size=BATCH_SIZE, 
                     epochs=PRUNING_EPOCHS, 
                     number_of_clusters=128)

weight_cluster_model(model_path='flowers_models_optimized/EfficentNetB0_flowers_model_ConstantSparsity90.h5', 
                     ds_train=flowers_datasets[0], 
                     ds_validation=flowers_datasets[1], 
                     batch_size=BATCH_SIZE, 
                     epochs=PRUNING_EPOCHS, 
                     number_of_clusters=32)
weight_cluster_model(model_path='flowers_models_optimized/EfficentNetB0_flowers_model_ConstantSparsity90.h5', 
                     ds_train=flowers_datasets[0], 
                     ds_validation=flowers_datasets[1], 
                     batch_size=BATCH_SIZE, 
                     epochs=PRUNING_EPOCHS, 
                     number_of_clusters=128)

weight_cluster_model(model_path='flowers_models_optimized/EfficentNetB0_flowers_model_PolynomialDecay90.h5', 
                     ds_train=flowers_datasets[0], 
                     ds_validation=flowers_datasets[1], 
                     batch_size=BATCH_SIZE, 
                     epochs=PRUNING_EPOCHS, 
                     number_of_clusters=32)
weight_cluster_model(model_path='flowers_models_optimized/EfficentNetB0_flowers_model_PolynomialDecay90.h5', 
                     ds_train=flowers_datasets[0], 
                     ds_validation=flowers_datasets[1], 
                     batch_size=BATCH_SIZE, 
                     epochs=PRUNING_EPOCHS, 
                     number_of_clusters=128)

#### beans models

In [ ]:
weight_cluster_model(model_path='beans_models_optimized/MobileNetV2_beans_model_ConstantSparsity50.h5', 
                     ds_train=beans_datasets[0], 
                     ds_validation=beans_datasets[1], 
                     batch_size=BATCH_SIZE, 
                     epochs=PRUNING_EPOCHS, 
                     number_of_clusters=32)
weight_cluster_model(model_path='beans_models_optimized/MobileNetV2_beans_model_ConstantSparsity50.h5', 
                     ds_train=beans_datasets[0], 
                     ds_validation=beans_datasets[1], 
                     batch_size=BATCH_SIZE, 
                     epochs=PRUNING_EPOCHS, 
                     number_of_clusters=128)

weight_cluster_model(model_path='beans_models_optimized/MobileNetV2_beans_model_PolynomialDecay50.h5', 
                     ds_train=beans_datasets[0], 
                     ds_validation=beans_datasets[1], 
                     batch_size=BATCH_SIZE, 
                     epochs=PRUNING_EPOCHS, 
                     number_of_clusters=32)
weight_cluster_model(model_path='beans_models_optimized/MobileNetV2_beans_model_PolynomialDecay50.h5', 
                     ds_train=beans_datasets[0], 
                     ds_validation=beans_datasets[1], 
                     batch_size=BATCH_SIZE, 
                     epochs=PRUNING_EPOCHS, 
                     number_of_clusters=128)

weight_cluster_model(model_path='beans_models_optimized/MobileNetV2_beans_model_ConstantSparsity75.h5', 
                     ds_train=beans_datasets[0], 
                     ds_validation=beans_datasets[1], 
                     batch_size=BATCH_SIZE, 
                     epochs=PRUNING_EPOCHS, 
                     number_of_clusters=32)
weight_cluster_model(model_path='beans_models_optimized/MobileNetV2_beans_model_ConstantSparsity75.h5', 
                     ds_train=beans_datasets[0], 
                     ds_validation=beans_datasets[1], 
                     batch_size=BATCH_SIZE, 
                     epochs=PRUNING_EPOCHS, 
                     number_of_clusters=128)

weight_cluster_model(model_path='beans_models_optimized/MobileNetV2_beans_model_PolynomialDecay75.h5', 
                     ds_train=beans_datasets[0], 
                     ds_validation=beans_datasets[1], 
                     batch_size=BATCH_SIZE, 
                     epochs=PRUNING_EPOCHS, 
                     number_of_clusters=32)
weight_cluster_model(model_path='beans_models_optimized/MobileNetV2_beans_model_PolynomialDecay75.h5', 
                     ds_train=beans_datasets[0], 
                     ds_validation=beans_datasets[1], 
                     batch_size=BATCH_SIZE, 
                     epochs=PRUNING_EPOCHS, 
                     number_of_clusters=128)

weight_cluster_model(model_path='beans_models_optimized/MobileNetV2_beans_model_ConstantSparsity90.h5', 
                     ds_train=beans_datasets[0], 
                     ds_validation=beans_datasets[1], 
                     batch_size=BATCH_SIZE, 
                     epochs=PRUNING_EPOCHS, 
                     number_of_clusters=32)
weight_cluster_model(model_path='beans_models_optimized/MobileNetV2_beans_model_ConstantSparsity90.h5', 
                     ds_train=beans_datasets[0], 
                     ds_validation=beans_datasets[1], 
                     batch_size=BATCH_SIZE, 
                     epochs=PRUNING_EPOCHS, 
                     number_of_clusters=128)

weight_cluster_model(model_path='beans_models_optimized/MobileNetV2_beans_model_PolynomialDecay90.h5', 
                     ds_train=beans_datasets[0], 
                     ds_validation=beans_datasets[1], 
                     batch_size=BATCH_SIZE, 
                     epochs=PRUNING_EPOCHS, 
                     number_of_clusters=32)
weight_cluster_model(model_path='beans_models_optimized/MobileNetV2_beans_model_PolynomialDecay90.h5', 
                     ds_train=beans_datasets[0], 
                     ds_validation=beans_datasets[1], 
                     batch_size=BATCH_SIZE, 
                     epochs=PRUNING_EPOCHS, 
                     number_of_clusters=128)

In [ ]:
weight_cluster_model(model_path='beans_models_optimized/EfficentNetB0_beans_model_ConstantSparsity50.h5', 
                     ds_train=beans_datasets[0], 
                     ds_validation=beans_datasets[1], 
                     batch_size=BATCH_SIZE, 
                     epochs=PRUNING_EPOCHS, 
                     number_of_clusters=32)
weight_cluster_model(model_path='beans_models_optimized/EfficentNetB0_beans_model_ConstantSparsity50.h5', 
                     ds_train=beans_datasets[0], 
                     ds_validation=beans_datasets[1], 
                     batch_size=BATCH_SIZE, 
                     epochs=PRUNING_EPOCHS, 
                     number_of_clusters=128)

weight_cluster_model(model_path='beans_models_optimized/EfficentNetB0_beans_model_PolynomialDecay50.h5', 
                     ds_train=beans_datasets[0], 
                     ds_validation=beans_datasets[1], 
                     batch_size=BATCH_SIZE, 
                     epochs=PRUNING_EPOCHS, 
                     number_of_clusters=32)
weight_cluster_model(model_path='beans_models_optimized/EfficentNetB0_beans_model_PolynomialDecay50.h5', 
                     ds_train=beans_datasets[0], 
                     ds_validation=beans_datasets[1], 
                     batch_size=BATCH_SIZE, 
                     epochs=PRUNING_EPOCHS, 
                     number_of_clusters=128)

weight_cluster_model(model_path='beans_models_optimized/EfficentNetB0_beans_model_ConstantSparsity75.h5', 
                     ds_train=beans_datasets[0], 
                     ds_validation=beans_datasets[1], 
                     batch_size=BATCH_SIZE, 
                     epochs=PRUNING_EPOCHS, 
                     number_of_clusters=32)
weight_cluster_model(model_path='beans_models_optimized/EfficentNetB0_beans_model_ConstantSparsity75.h5', 
                     ds_train=beans_datasets[0], 
                     ds_validation=beans_datasets[1], 
                     batch_size=BATCH_SIZE, 
                     epochs=PRUNING_EPOCHS, 
                     number_of_clusters=128)

weight_cluster_model(model_path='beans_models_optimized/EfficentNetB0_beans_model_PolynomialDecay75.h5', 
                     ds_train=beans_datasets[0], 
                     ds_validation=beans_datasets[1], 
                     batch_size=BATCH_SIZE, 
                     epochs=PRUNING_EPOCHS, 
                     number_of_clusters=32)
weight_cluster_model(model_path='beans_models_optimized/EfficentNetB0_beans_model_PolynomialDecay75.h5', 
                     ds_train=beans_datasets[0], 
                     ds_validation=beans_datasets[1], 
                     batch_size=BATCH_SIZE, 
                     epochs=PRUNING_EPOCHS, 
                     number_of_clusters=128)

weight_cluster_model(model_path='beans_models_optimized/EfficentNetB0_beans_model_ConstantSparsity90.h5', 
                     ds_train=beans_datasets[0], 
                     ds_validation=beans_datasets[1], 
                     batch_size=BATCH_SIZE, 
                     epochs=PRUNING_EPOCHS, 
                     number_of_clusters=32)
weight_cluster_model(model_path='beans_models_optimized/EfficentNetB0_beans_model_ConstantSparsity90.h5', 
                     ds_train=beans_datasets[0], 
                     ds_validation=beans_datasets[1], 
                     batch_size=BATCH_SIZE, 
                     epochs=PRUNING_EPOCHS, 
                     number_of_clusters=128)

weight_cluster_model(model_path='beans_models_optimized/EfficentNetB0_beans_model_PolynomialDecay90.h5', 
                     ds_train=beans_datasets[0], 
                     ds_validation=beans_datasets[1], 
                     batch_size=BATCH_SIZE, 
                     epochs=PRUNING_EPOCHS, 
                     number_of_clusters=32)
weight_cluster_model(model_path='beans_models_optimized/EfficentNetB0_beans_model_PolynomialDecay90.h5', 
                     ds_train=beans_datasets[0], 
                     ds_validation=beans_datasets[1], 
                     batch_size=BATCH_SIZE, 
                     epochs=PRUNING_EPOCHS, 
                     number_of_clusters=128)

## Quantization of all models

In [20]:
# get all models paths
model_paths = []
import os
for file in os.listdir("beans_models/"):
    if file.endswith(".h5"):
        model_paths.append(str(os.path.join("beans_models/", file)))
for file in os.listdir("beans_models_optimized/"):
    if file.endswith(".h5"):
        model_paths.append(str(os.path.join("beans_models_optimized/", file)))
for file in os.listdir("flowers_models/"):
    if file.endswith(".h5"):
        model_paths.append(str(os.path.join("flowers_models/", file)))
for file in os.listdir("flowers_models_optimized/"):
    if file.endswith(".h5"):
        model_paths.append(str(os.path.join("flowers_models_optimized/", file)))

In [21]:
model_paths

['beans_models/MobileNetV2_beans_model.h5',
 'beans_models/EfficentNetB0_beans_model.h5',
 'beans_models_optimized/MobileNetV2_beans_model_KMeansPlusPlus32_ConstantSparsity90.h5',
 'beans_models_optimized/MobileNetV2_beans_model_KMeansPlusPlus128_PolynomialDecay75.h5',
 'beans_models_optimized/MobileNetV2_beans_model_PolynomialDecay75.h5',
 'beans_models_optimized/MobileNetV2_beans_model_KMeansPlusPlus128.h5',
 'beans_models_optimized/MobileNetV2_beans_model_KMeansPlusPlus128_PolynomialDecay50.h5',
 'beans_models_optimized/MobileNetV2_beans_model_PolynomialDecay50.h5',
 'beans_models_optimized/MobileNetV2_beans_model_KMeansPlusPlus32_PolynomialDecay90.h5',
 'beans_models_optimized/MobileNetV2_beans_model_KMeansPlusPlus128_ConstantSparsity90.h5',
 'beans_models_optimized/MobileNetV2_beans_model_ConstantSparsity50.h5',
 'beans_models_optimized/MobileNetV2_beans_model_ConstantSparsity75.h5',
 'beans_models_optimized/MobileNetV2_beans_model_KMeansPlusPlus128_ConstantSparsity75.h5',
 'beans

In [22]:
for model_path in model_paths:
    if "flowers" in model_path:
        model_quantization(model_path=model_path, ds=flowers_datasets[0])
    if "beans" in model_path:
        model_quantization(model_path=model_path, ds=beans_datasets[0])

INFO:tensorflow:Assets written to: /var/folders/_1/7lg8klcj1d55272nzrt0k5740000gn/T/tmpnd_a_qrd/assets


INFO:tensorflow:Assets written to: /var/folders/_1/7lg8klcj1d55272nzrt0k5740000gn/T/tmpnd_a_qrd/assets


Converted TFLite model (8873924 Bytes) saved to: beans_models_optimized/MobileNetV2_beans_model.tflite
INFO:tensorflow:Assets written to: /var/folders/_1/7lg8klcj1d55272nzrt0k5740000gn/T/tmpcxlpw9j4/assets


INFO:tensorflow:Assets written to: /var/folders/_1/7lg8klcj1d55272nzrt0k5740000gn/T/tmpcxlpw9j4/assets


Dynamic range quantizatized TFLite model (2641872 Bytes) saved to: beans_models_optimized/MobileNetV2_beans_model_dynamic_rage_quantization.tflite
INFO:tensorflow:Assets written to: /var/folders/_1/7lg8klcj1d55272nzrt0k5740000gn/T/tmp8k2_ebr9/assets


...

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3417, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-22-7097d41c5c42>", line 3, in <module>
    model_quantization(model_path=model_path, ds=flowers_datasets[0])
  File "<ipython-input-6-8b0f13698ff5>", line 63, in model_quantization
    tflite_model_quant = converter.convert()
  File "/usr/local/lib/python3.8/site-packages/tensorflow/lite/python/lite.py", line 829, in convert
    saved_model_convert_result = self._convert_as_saved_model()
  File "/usr/local/lib/python3.8/site-packages/tensorflow/lite/python/lite.py", line 782, in _convert_as_saved_model
    self._keras_model.save(temp_dir, save_format="tf")
  File "/usr/local/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py", line 2001, in save
    save.save_model(self, filepath, overwrite, include_optimizer, save_format,
  File "/usr/local/lib/python3

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3417, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-22-7097d41c5c42>", line 3, in <module>
    model_quantization(model_path=model_path, ds=flowers_datasets[0])
  File "<ipython-input-6-8b0f13698ff5>", line 63, in model_quantization
    tflite_model_quant = converter.convert()
  File "/usr/local/lib/python3.8/site-packages/tensorflow/lite/python/lite.py", line 829, in convert
    saved_model_convert_result = self._convert_as_saved_model()
  File "/usr/local/lib/python3.8/site-packages/tensorflow/lite/python/lite.py", line 782, in _convert_as_saved_model
    self._keras_model.save(temp_dir, save_format="tf")
  File "/usr/local/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py", line 2001, in save
    save.save_model(self, filepath, overwrite, include_optimizer, save_format,
  File "/usr/local/lib/python3


KeyboardInterrupt

